In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error
import pandas as pd

In [2]:
N = 10000000

In [3]:
df = pd.read_csv("trial_data.csv", header = 0) 
df.head()

,Date,Uncovered,Du,Ru,Exposed,Isolated,Apathetic,Da,Ra,Infected,Di,Ri
0,24-03-20,39,6,33,557,10,11,2,9,536,14,522
1,25-03-20,57,10,47,692,12,23,4,19,657,33,624
2,26-03-20,71,13,58,781,18,34,6,28,729,69,660
3,27-03-20,83,11,72,946,20,39,8,31,887,109,778
4,28-03-20,97,15,82,1057,24,46,9,37,987,144,843


In [4]:
available = len(df.index)
print("Available", available, "days")

Available 52 days


In [5]:
start_date = df['Date'].iloc[0]  
uncovered = df['Uncovered'].values.tolist() 
Du = df['Du'].values.tolist() # shob gula ek ekta col k tar nijeder moddhe rakhlam.. 
Ru = df['Ru'].values.tolist() # shob gula ek ekta col k tar nijeder moddhe rakhlam..
exposed= df['Exposed'].values.tolist()
isolated = df['Isolated'].values.tolist()
infected = df['Infected'].values.tolist()  # shob gula ek ekta col k tar nijeder moddhe rakhlam..
Di = df['Di'].values.tolist()
Ri = df['Ri'].values.tolist()
Apathetic = df['Apathetic'].values.tolist() 
Da = df['Da'].values.tolist() # shob gula ek ekta col k tar nijeder moddhe rakhlam.. 
Ra = df['Ra'].values.tolist() # shob gula ek ekta col k tar nijeder moddhe rakhlam..


In [6]:
print(start_date)

24-03-20


In [7]:
#Helper Function

t_max = available + 21

date = np.array('2020-03-24', dtype=np.datetime64) 
dates = date + np.arange(t_max)
str_dates = []
for i in dates:
    str_dates.append(str(i))
print(str_dates)
print(len(str_dates))

['2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06', '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-11', '2020-04-12', '2020-04-13', '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17', '2020-04-18', '2020-04-19', '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-23', '2020-04-24', '2020-04-25', '2020-04-26', '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30', '2020-05-01', '2020-05-02', '2020-05-03', '2020-05-04', '2020-05-05', '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-09', '2020-05-10', '2020-05-11', '2020-05-12', '2020-05-13', '2020-05-14', '2020-05-15', '2020-05-16', '2020-05-17', '2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21', '2020-05-22', '2020-05-23', '2020-05-24', '2020-05-25', '2020-05-26', '2020-05-27', '2020-05-28', '2020-05-29', '2020-05-30', '2020-05-31', '2020-06-01', '2020-06-02', '2020

In [8]:
def seird_model(init_vals, params, t):

    DATE_0, S_0, U_0, Du_0, Ru_0, E_0, Is_0, I_0,  Di_0, Ri_0, A_0, Da_0, Ra_0 = init_vals  # jehetu ajk r real data niye kaj kortesi tai date takeo mention korte hobe.
    DATE, S, U, Du, Ru, E, Is, I, Di, Ri, A, Da, Ra = [DATE_0] , [S_0] , [U_0] , [Du_0], [Ru_0] , [E_0] , [Is_0] , [I_0] ,  [Di_0] , [Ri_0], [A_0], [Da_0] , [Ra_0]
    
    beta, sigma, tao, alpha, rho, gamma, delta, miu, lamda, womega, eta  = params
    
    ##### Create next t days ######
    date = np.array(DATE, dtype=np.datetime64)
    dates = date + np.arange(len(t))
    str_dates = []
    for i in dates:
        str_dates.append(str(i))
        
    ##### End creating t days #####
    for tic in t[1:]:                # 1st din er initial value deya ache tai 1 theke shuru kore baki shob add dilam.

        DATE.append(str_dates[tic])

        
        next_S = S[-1] - ( alpha * (S[-1]  * E[-1] / N ) ) - ( beta * (S[-1]  * U[-1] / N ) )  # Susceptible
        next_U = U[-1] + ( beta * (S[-1]  * U[-1] / N ) ) - ( sigma * U[-1] ) - ( tao * U[-1] )  # Unknown
        next_E = E[-1] + ( alpha * (S[-1]  * E[-1] / N ) ) -  ( gamma * I[-1] ) - ( lamda * A[-1] ) - ( rho * Is[-1] ) # Exposed
        next_Du = Du[-1] + ( sigma * U[-1] )
        next_Ru = Ru[-1] + ( tao * U[-1]) 
        next_Di = Di[-1] + ( delta * I[-1] )  
        next_Ri = Ri[-1] + ( miu * I[-1] )
        next_A = A[-1] + ( lamda * A[-1] ) - ( womega * A[-1] ) - ( miu * A[-1] )  # Unknown 
        next_Da = Da[-1] + ( womega * U[-1] )
        next_Ra = Ra[-1] + ( eta * U[-1] )
        next_Is = Is[-1] + (rho *  Is[-1] )  #isolation
        next_I = I[-1] + ( gamma * I[-1] ) - ( delta * I[-1] ) - ( miu * I[-1] ) # Infected
       
        
        

        
        S.append(next_S)
        U.append(next_U)
        E.append(next_E)
        Du.append(next_Du)                   
        Ru.append(next_Ru)
        Di.append(next_Di)
        Ri.append(next_Ri)
        A.append(next_A)
        Da.append(next_Da)
        Ra.append(next_Ra)
        Is.append(next_Is)
        I.append(next_I)
        
        

    return np.stack([DATE, S, U, E, Du, Ru, Di, Ri, A, Da, Ra, Is, I]).T

In [9]:
# Spliting the data for one day rolling window approach

train_min = 1  # diffrence ber korar jnno ami 1 din er diffrence k niye cal kortesi
train_max = available + 1

uncovered_train = []
Du_train = []
Ru_train = []
exposed_train = []
isolated_train = []
infected_train = []
Di_train = []
Ri_train = []
Apathetic_train = []
Da_train = []
Ra_train = []


for i in range(train_min+1, train_max):
    j = i - 2
    uncovered_train.append(uncovered[j:i])
    Du_train.append(Du[j:i])
    Ru_train.append(Ru[j:i])
    exposed_train.append(exposed[j:i])
    isolated_train.append(isolated[j:i])
    infected_train.append(infected[j:i])
    Di_train.append(Di[j:i])
    Ri_train.append(Ri[j:i])
    Apathetic_train.append(Apathetic[j:i])
    Da_train.append(Da[j:i])
    Ra_train.append(Ra[j:i])

In [10]:
for i in range(len(exposed_train)): 
    print(i,exposed_train[i])

0 [557, 692]
1 [692, 781]
2 [781, 946]
3 [946, 1057]
4 [1057, 1102]
5 [1102, 1337]
6 [1337, 1488]
7 [1488, 2114]
8 [2114, 2678]
9 [2678, 2705]
10 [2705, 3254]
11 [3254, 3778]
12 [3778, 5049]
13 [5049, 5615]
14 [5615, 6303]
15 [6303, 7234]
16 [7234, 8237]
17 [8237, 9280]
18 [9280, 10157]
19 [10157, 11468]
20 [11468, 12663]
21 [12663, 13542]
22 [13542, 14974]
23 [14974, 16160]
24 [16160, 17919]
25 [17919, 20146]
26 [20146, 21476]
27 [21476, 23695]
28 [23695, 25351]
29 [25351, 27655]
30 [27655, 29580]
31 [29580, 31732]
32 [31732, 33898]
33 [33898, 36054]
34 [36054, 38511]
35 [38511, 40924]
36 [40924, 43341]
37 [43341, 46623]
38 [46623, 49855]
39 [49855, 53545]
40 [53545, 58527]
41 [58527, 62764]
42 [62764, 67453]
43 [67453, 72197]
44 [72197, 76581]
45 [76581, 81068]
46 [81068, 85025]
47 [85025, 90215]
48 [90215, 96068]
49 [96068, 102657]
50 [102657, 108097]


In [11]:
t2 = np.arange(0, 10, 1)   # 0 - 3 PORJNTO JABE.. R PROTTEKBAR 1 KORE KORE BARBE. 0 initial value tar mane 0 theke shuRa hoye aro 2 din porjnto simulation cholbe.
print(t2)

[0 1 2 3 4 5 6 7 8 9]


In [12]:
last5_vals = []
last5_params = []


t_incub = 21  # Assumption 30 days

for sample in range(len(uncovered_train)):
    
    u_0 = uncovered_train[sample][0]
    u_1 = uncovered_train[sample][1]
    u_diff = u_1 - u_0

    du_0 = Du_train[sample][0]
    du_1 = Du_train[sample][1]
    du_diff = du_1 - du_0

    ru_0 = Ru_train[sample][0]
    ru_1 = Ru_train[sample][1]
    ru_diff = ru_1 - ru_0
    
    e_0 = exposed_train[sample][0]
    e_1 = exposed_train[sample][1]
    e_diff = e_1 - e_0
    
    is_0 = isolated_train[sample][0]
    is_1 = isolated_train[sample][1]
    is_diff = is_1 - is_0

    i_0 = infected_train[sample][0]
    i_1 = infected_train[sample][1]
    i_diff = i_1 - i_0
    
    di_0 = Di_train[sample][0]
    di_1 = Di_train[sample][1]
    di_diff = di_1 - di_0
    
    ri_0 = Ri_train[sample][0]
    ri_1 = Ri_train[sample][1]
    ri_diff = ri_1 - ri_0
    
    
    a_0 = Apathetic_train[sample][0]
    a_1 = Apathetic_train[sample][1]
    a_diff = a_1 - a_0

    Da_0 = Da_train[sample][0]
    Da_1 = Da_train[sample][1]
    Da_diff = Da_1 - Da_0

    Ra_0 = Ra_train[sample][0]
    Ra_1 = Ra_train[sample][1]
    Ra_diff = Ra_1 - Ra_0

  
    s_0 = N - du_0 - ru_0 - is_0 - di_0 - ri_0 - Da_0 - Ra_0 
    
    beta =   (( u_diff + du_diff + ru_diff ) / (s_0 * u_0 )) * N     # beta
    sigma = ( du_diff / u_0 )     # sigma
    tao = ( ru_diff / u_0 )      # tao
    alpha = (( e_diff + is_diff + i_diff + di_diff + ri_diff + a_diff + Da_diff + Ra_diff ) / (s_0 * e_0 ) )* N  # alpha
    rho = ( is_diff / is_0 )      # rho
    gamma = ( i_diff + di_diff + ri_diff) / i_0    # gamma
    delta = ( di_diff / i_0 )   # delta
    miu =  ( ri_diff / i_0 )     # miu
    lamda = (a_diff + Da_diff + Ra_diff ) / a_0
    womega = (Da_diff / a_0)
    eta =  (Ra_diff / a_0)
    

    init_vals = str_dates[sample], s_0, u_0, du_0, ru_0, e_0, is_0, i_0, di_0, ri_0, a_0, Da_0, Ra_0  
    params = beta, sigma, tao, alpha, rho, gamma, delta, miu, lamda, womega, eta 
    pred = seird_model(init_vals, params, t2) # t2 cz etar upor porer day gula depend kortese.
    

    # Store Last five cases for predecting future
    if ((len(uncovered_train)  - sample ) <= 15):
        print(str_dates[sample])
        last5_vals.append(init_vals)
        last5_params.append(params)

2020-04-29
2020-04-30
2020-05-01
2020-05-02
2020-05-03
2020-05-04
2020-05-05
2020-05-06
2020-05-07
2020-05-08
2020-05-09
2020-05-10
2020-05-11
2020-05-12
2020-05-13


In [13]:
for v in last5_vals: 
    print(v)

('2020-04-29', 9950639, 8437, 1265, 7172, 40924, 1079, 33062, 4955, 28107, 6783, 949, 5834)
('2020-04-30', 9947591, 9068, 1321, 7747, 43341, 1154, 34863, 5077, 29786, 7324, 991, 6333)
('2020-05-01', 9943370, 10007, 1408, 8599, 46623, 1223, 37257, 5296, 31961, 8143, 1006, 7137)
('2020-05-02', 9939326, 10819, 1593, 9226, 49855, 1323, 39699, 5843, 33856, 8833, 1195, 7638)
('2020-05-03', 9934680, 11775, 1760, 10015, 53545, 1391, 42505, 6353, 36152, 9649, 1320, 8329)
('2020-05-04', 9928626, 12847, 1878, 10969, 58527, 1566, 46437, 6785, 39652, 10524, 1409, 9115)
('2020-05-05', 9923094, 14142, 2122, 12020, 62764, 1693, 49400, 7410, 41990, 11671, 1592, 10079)
('2020-05-06', 9917216, 15331, 2291, 13040, 67453, 1785, 52987, 7916, 45071, 12681, 1719, 10962)
('2020-05-07', 9911027, 16776, 2514, 14262, 72197, 1889, 56351, 8444, 47907, 13957, 1886, 12071)
('2020-05-08', 9905532, 17887, 2680, 15207, 76581, 1985, 59695, 8944, 50751, 14901, 2010, 12891)
('2020-05-09', 9898839, 19301, 2796, 16505, 81068

In [14]:
for p in last5_params: 
    print(p)

(0.150321234972947, 0.0066374303662439255, 0.0681521867962546, 0.17621926411799912, 0.06950880444856349, 0.10894682717318976, 0.0036900369003690036, 0.05078337668622588, 0.15951643815420904, 0.006191950464396285, 0.07356626861270824)
(0.20819301555510808, 0.009594177326863696, 0.09395677106307895, 0.22677165361466547, 0.05979202772963605, 0.13733757852164186, 0.006281731348420962, 0.062387057912399964, 0.22364827962861825, 0.0020480611687602405, 0.10977607864554888)
(0.16321066149638983, 0.01848705905865894, 0.06265614070150895, 0.2069933646534687, 0.08176614881439084, 0.13108945969884853, 0.014681804761521325, 0.050862925087902945, 0.16947071104015718, 0.02321011912071718, 0.061525236399361416)
(0.17780494367179214, 0.015435807375912746, 0.07292725760236621, 0.22202709913988516, 0.05139833711262283, 0.14136376231139322, 0.012846671200785914, 0.057835209954910705, 0.18476168912034416, 0.014151477414242046, 0.07822936714593004)
(0.18327785032383548, 0.010021231422505307, 0.0810191082802

In [15]:
t7 = np.arange(0, 21, 1)  #13 din er data ashbe
print(t7)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]


In [16]:
for z in range(1, len(last5_params)+1):
    results = seird_model(last5_vals[z-1], last5_params[z-1], t7)
    t_z = np.arange(7 - z, 22 - z, 1)
    print("Predictions using", last5_vals[z-1][0], "parameters\n")
    print("---------------------------------------------------------------------------------------------------------------------------")
    print("Date", " "," "," " "Uncovered", " " "Exposed" , " " "Du", " "," "," " "Ru", " ", " " "Di", " ", " ", " " "Ri", " " "Apathetic", " " "Da", " ", " ", "Ra", " " "Isolated", " " "Infected")
    print("---------------------------------------------------------------------------------------------------------------------------")
    for day in t_z:
        print(results[day][0]," ",int(float(results[day][2]))," ",int(float(results[day][3]))," ", int(float(results[day][4]))," ", int(float(results[day][5])), " ", int(float(results[day][6]))," ",int(float(results[day][7]))," ",int(float(results[day][8]))," ",int(float(results[day][9])), " ", int(float(results[day][10])), " ", int(float(results[day][11])), " ", int(float(results[day][12])))
    print("---------------------------------------------------------------------------------------------------------------\n\n")

Predictions using 2020-04-29 parameters

---------------------------------------------------------------------------------------------------------------------------
Date      Uncovered  Exposed  Du      Ru    Di      Ri  Apathetic  Da     Ra  Isolated  Infected
---------------------------------------------------------------------------------------------------------------------------
2020-05-05   12980   56851   1670   11333   5794   39656   12184   1327   10325   1614   45450
2020-05-06   13939   59753   1756   12217   5961   41964   13433   1407   11280   1727   47926
2020-05-07   14967   62672   1848   13167   6138   44398   14810   1493   12306   1847   50537
2020-05-08   16068   65573   1948   14187   6325   46965   16329   1586   13407   1975   53290
2020-05-09   17247   68412   2054   15283   6521   49671   18004   1685   14589   2112   56193
2020-05-10   18508   71135   2169   16458   6729   52525   19850   1792   15858   2259   59254
2020-05-11   19858   73673   2292   17719   

2020-05-14   30076   136034   3830   26751   14041   98445   24848   3225   25156   5121   112487
2020-05-15   32652   146488   4131   29188   15185   107708   27079   3502   27606   5766   122893
2020-05-16   35424   157159   4458   31833   16434   117828   29511   3803   30266   6491   134262
2020-05-17   38402   167843   4813   34703   17798   128883   32161   4130   33151   7308   146682
2020-05-18   41596   178257   5198   37814   19289   140961   35049   4484   36279   8228   160251
2020-05-19   45018   188018   5615   41184   20918   154157   38196   4868   39668   9263   175075
---------------------------------------------------------------------------------------------------------------


Predictions using 2020-05-04 parameters

---------------------------------------------------------------------------------------------------------------------------
Date      Uncovered  Exposed  Du      Ru    Di      Ri  Apathetic  Da     Ra  Isolated  Infected
-------------------------------

2020-05-17   42220   116432   4265   38275   13424   70624   27701   3664   24148   3752   90740
2020-05-18   46480   121312   4543   42390   14064   73110   29631   3970   26045   4034   95011
2020-05-19   51147   126228   4849   46920   14734   75714   31697   4306   28133   4338   99482
---------------------------------------------------------------------------------------------------------------


Predictions using 2020-05-10 parameters

---------------------------------------------------------------------------------------------------------------------------
Date      Uncovered  Exposed  Du      Ru    Di      Ri  Apathetic  Da     Ra  Isolated  Infected
---------------------------------------------------------------------------------------------------------------------------
2020-05-26   39370   209427   6579   33512   17144   154981   36488   6010   37615   5833   172125
2020-05-27   40828   219929   6889   34796   17892   164900   38272   6336   39549   6190   182793
2020-05-28 